In [1]:
import ee
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'fcntl'

visit: https://www.isric.org/explore/soilgrids/faq-soilgrids#How_can_I_access_SoilGrids

visit: http://www.css.cornell.edu/faculty/dgr2/_static/files/R_html/SoilGridsPCA.html#5_Stack_and_standardize

visit: https://developers.google.com/earth-engine/datasets/catalog/WORLDCLIM_V1_MONTHLY#bands

In [54]:
ee.Authenticate()


Successfully saved authorization token.


In [55]:
ee.Initialize()

In [56]:
# Area de interes
pois = pd.read_csv('puntos.csv')
pois= pois.drop(["index"], 1)

C:\ProgramData\Anaconda3\envs\tesis\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [57]:
poi = ee.Geometry.Point([pois.x[0], pois.y[0]])

In [58]:
SGc = ee.Image("projects/soilgrids-isric/clay_mean").clip(poi)   # clay #
SGsi = ee.Image("projects/soilgrids-isric/silt_mean").clip(poi)   # silt #
SGn = ee.Image("projects/soilgrids-isric/nitrogen_mean").clip(poi)   # nitrogen #
SGs = ee.Image("projects/soilgrids-isric/sand_mean").clip(poi)   # sand #
SGbd = ee.Image("projects/soilgrids-isric/bdod_mean").clip(poi)  # bulk density #
SGsoc = ee.Image("projects/soilgrids-isric/soc_mean").clip(poi)  # SOC #
SGpH = ee.Image("projects/soilgrids-isric/phh2o_mean").clip(poi) # pH H2O #
SGcec = ee.Image("projects/soilgrids-isric/cec_mean").clip(poi)  # CEC #
SGcf = ee.Image("projects/soilgrids-isric/cfvo_mean").clip(poi)  # coarse fragments #

SG = SGc.addBands(SGs).addBands(SGbd).addBands(SGsoc).addBands(SGpH).addBands(SGcec).addBands(SGcf).addBands(SGn).addBands(SGsi)

In [59]:
metadata = SG.propertyNames

In [60]:
inputBandNames = SG.bandNames().getInfo(); inputBandNames

['clay_0-5cm_mean',
 'clay_5-15cm_mean',
 'clay_15-30cm_mean',
 'clay_30-60cm_mean',
 'clay_60-100cm_mean',
 'clay_100-200cm_mean',
 'sand_0-5cm_mean',
 'sand_5-15cm_mean',
 'sand_15-30cm_mean',
 'sand_30-60cm_mean',
 'sand_60-100cm_mean',
 'sand_100-200cm_mean',
 'bdod_0-5cm_mean',
 'bdod_5-15cm_mean',
 'bdod_15-30cm_mean',
 'bdod_30-60cm_mean',
 'bdod_60-100cm_mean',
 'bdod_100-200cm_mean',
 'soc_0-5cm_mean',
 'soc_5-15cm_mean',
 'soc_15-30cm_mean',
 'soc_30-60cm_mean',
 'soc_60-100cm_mean',
 'soc_100-200cm_mean',
 'phh2o_0-5cm_mean',
 'phh2o_5-15cm_mean',
 'phh2o_15-30cm_mean',
 'phh2o_30-60cm_mean',
 'phh2o_60-100cm_mean',
 'phh2o_100-200cm_mean',
 'cec_0-5cm_mean',
 'cec_5-15cm_mean',
 'cec_15-30cm_mean',
 'cec_30-60cm_mean',
 'cec_60-100cm_mean',
 'cec_100-200cm_mean',
 'cfvo_0-5cm_mean',
 'cfvo_5-15cm_mean',
 'cfvo_15-30cm_mean',
 'cfvo_30-60cm_mean',
 'cfvo_60-100cm_mean',
 'cfvo_100-200cm_mean',
 'nitrogen_0-5cm_mean',
 'nitrogen_5-15cm_mean',
 'nitrogen_15-30cm_mean',
 'nitro

In [61]:
dimOne = len(inputBandNames)
print("Number of input bands:", dimOne)

Number of input bands: 54


In [62]:
proj = SG.select("bdod_0-5cm_mean").projection()
proj.getInfo()

{'type': 'Projection',
 'wkt': 'PROJCS["World_Mollweide", \n  GEOGCS["GCS_WGS_1984", \n    DATUM["D_WGS_1984", \n      SPHEROID["WGS_1984", 6378137.0, 298.257223563]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH]], \n  PROJECTION["Mollweide"], \n  PARAMETER["semi_minor", 6378137.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  PARAMETER["central_meridian", 0.0], \n  UNIT["m", 1.0], \n  AXIS["x", EAST], \n  AXIS["y", NORTH]]',
 'transform': [250, 0, -17960089.445510544, 0, -250, 8697788.131638981]}

In [63]:
scale = SG.select("bdod_0-5cm_mean").projection().nominalScale()
scale.getInfo()

250

In [64]:
SGmean = SG.reduceRegion(ee.Reducer.mean(),geometry = poi, scale = scale, bestEffort = True)

In [65]:
resultado = pd.DataFrame(SGmean.getInfo(), index=[0])
resultado

,bdod_0-5cm_mean,bdod_100-200cm_mean,bdod_15-30cm_mean,bdod_30-60cm_mean,bdod_5-15cm_mean,bdod_60-100cm_mean,cec_0-5cm_mean,cec_100-200cm_mean,cec_15-30cm_mean,cec_30-60cm_mean,...,silt_15-30cm_mean,silt_30-60cm_mean,silt_5-15cm_mean,silt_60-100cm_mean,soc_0-5cm_mean,soc_100-200cm_mean,soc_15-30cm_mean,soc_30-60cm_mean,soc_5-15cm_mean,soc_60-100cm_mean
0,129,133,133,134,133,132,244,208,227,218,...,383,364,405,378,425,50,219,88,334,61


In [66]:
# Ahora una función que calcule para todos los puntos:
data_set = []
for i in range(len(pois)): 
    poi = ee.Geometry.Point([pois.x[i], pois.y[i]])
    SGc = ee.Image("projects/soilgrids-isric/clay_mean").clip(poi)   # clay #
    SGsi = ee.Image("projects/soilgrids-isric/silt_mean").clip(poi)   # silt #
    SGn = ee.Image("projects/soilgrids-isric/nitrogen_mean").clip(poi)   # nitrogen #
    SGs = ee.Image("projects/soilgrids-isric/sand_mean").clip(poi)   # sand #
    SGbd = ee.Image("projects/soilgrids-isric/bdod_mean").clip(poi)  # bulk density #
    SGsoc = ee.Image("projects/soilgrids-isric/soc_mean").clip(poi)  # SOC #
    SGpH = ee.Image("projects/soilgrids-isric/phh2o_mean").clip(poi) # pH H2O #
    SGcec = ee.Image("projects/soilgrids-isric/cec_mean").clip(poi)  # CEC #
    SGcf = ee.Image("projects/soilgrids-isric/cfvo_mean").clip(poi)  # coarse fragments #
    SG = SGc.addBands(SGs).addBands(SGbd).addBands(SGsoc).addBands(SGpH).addBands(SGcec).addBands(SGcf).addBands(SGn).addBands(SGsi)
    scale = SG.select("bdod_0-5cm_mean").projection().nominalScale()
    SGmean = SG.reduceRegion(ee.Reducer.mean(),geometry = poi, scale = scale, bestEffort = True)
    resultado = pd.DataFrame(SGmean.getInfo(), index=[i])
    data_set.append(resultado)

2 horas 6 minutos

In [67]:
data_set_soil_grids = pd.DataFrame(np.concatenate(data_set),columns = ['bdod_0-5cm_mean', 'bdod_100-200cm_mean', 'bdod_15-30cm_mean',
       'bdod_30-60cm_mean', 'bdod_5-15cm_mean', 'bdod_60-100cm_mean',
       'cec_0-5cm_mean', 'cec_100-200cm_mean', 'cec_15-30cm_mean',
       'cec_30-60cm_mean', 'cec_5-15cm_mean', 'cec_60-100cm_mean',
       'cfvo_0-5cm_mean', 'cfvo_100-200cm_mean', 'cfvo_15-30cm_mean',
       'cfvo_30-60cm_mean', 'cfvo_5-15cm_mean', 'cfvo_60-100cm_mean',
       'clay_0-5cm_mean', 'clay_100-200cm_mean', 'clay_15-30cm_mean',
       'clay_30-60cm_mean', 'clay_5-15cm_mean', 'clay_60-100cm_mean',
       'nitrogen_0-5cm_mean', 'nitrogen_100-200cm_mean',
       'nitrogen_15-30cm_mean', 'nitrogen_30-60cm_mean',
       'nitrogen_5-15cm_mean', 'nitrogen_60-100cm_mean', 'phh2o_0-5cm_mean',
       'phh2o_100-200cm_mean', 'phh2o_15-30cm_mean', 'phh2o_30-60cm_mean',
       'phh2o_5-15cm_mean', 'phh2o_60-100cm_mean', 'sand_0-5cm_mean',
       'sand_100-200cm_mean', 'sand_15-30cm_mean', 'sand_30-60cm_mean',
       'sand_5-15cm_mean', 'sand_60-100cm_mean', 'silt_0-5cm_mean',
       'silt_100-200cm_mean', 'silt_15-30cm_mean', 'silt_30-60cm_mean',
       'silt_5-15cm_mean', 'silt_60-100cm_mean', 'soc_0-5cm_mean',
       'soc_100-200cm_mean', 'soc_15-30cm_mean', 'soc_30-60cm_mean',
       'soc_5-15cm_mean', 'soc_60-100cm_mean'])

In [ ]:
data_set_soil_grids.to_excel('DB_soilgrids.xlsx')

In [72]:
data_set_soil_grids = pd.read_excel('DB_soilgrids.xlsx')

In [73]:
data_set_soil_grids

,Unnamed: 0,bdod_0-5cm_mean,bdod_100-200cm_mean,bdod_15-30cm_mean,bdod_30-60cm_mean,bdod_5-15cm_mean,bdod_60-100cm_mean,cec_0-5cm_mean,cec_100-200cm_mean,cec_15-30cm_mean,...,silt_15-30cm_mean,silt_30-60cm_mean,silt_5-15cm_mean,silt_60-100cm_mean,soc_0-5cm_mean,soc_100-200cm_mean,soc_15-30cm_mean,soc_30-60cm_mean,soc_5-15cm_mean,soc_60-100cm_mean
0,0,129.0,133.0,133.0,134.0,133.0,132.0,244.0,208.0,227.0,...,383.0,364.0,405.0,378.0,425.0,50.0,219.0,88.0,334.0,61.0
1,1,123.0,128.0,128.0,124.0,127.0,126.0,181.0,136.0,166.0,...,296.0,272.0,302.0,275.0,488.0,79.0,256.0,114.0,461.0,98.0
2,2,123.0,128.0,128.0,124.0,127.0,126.0,181.0,136.0,166.0,...,296.0,272.0,302.0,275.0,488.0,79.0,256.0,114.0,461.0,98.0
3,3,123.0,128.0,128.0,124.0,127.0,126.0,181.0,136.0,166.0,...,296.0,272.0,302.0,275.0,488.0,79.0,256.0,114.0,461.0,98.0
4,4,123.0,128.0,128.0,124.0,127.0,126.0,181.0,136.0,166.0,...,296.0,272.0,302.0,275.0,488.0,79.0,256.0,114.0,461.0,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29579,29579,130.0,140.0,135.0,139.0,133.0,139.0,212.0,172.0,197.0,...,382.0,357.0,410.0,342.0,519.0,91.0,233.0,111.0,394.0,88.0
29580,29580,130.0,140.0,135.0,139.0,133.0,139.0,212.0,172.0,197.0,...,382.0,357.0,410.0,342.0,519.0,91.0,233.0,111.0,394.0,88.0
29581,29581,130.0,140.0,135.0,139.0,133.0,139.0,212.0,172.0,197.0,...,382.0,357.0,410.0,342.0,519.0,91.0,233.0,111.0,394.0,88.0
29582,29582,130.0,140.0,135.0,139.0,133.0,139.0,212.0,172.0,197.0,...,382.0,357.0,410.0,342.0,519.0,91.0,233.0,111.0,394.0,88.0


In [74]:
data_set_soil_grids[['bdod_0-5cm_mean', 'bdod_100-200cm_mean', 'bdod_15-30cm_mean',
       'bdod_30-60cm_mean', 'bdod_5-15cm_mean', 'bdod_60-100cm_mean','nitrogen_0-5cm_mean', 'nitrogen_100-200cm_mean',
       'nitrogen_15-30cm_mean', 'nitrogen_30-60cm_mean',
       'nitrogen_5-15cm_mean', 'nitrogen_60-100cm_mean']] = data_set_soil_grids[['bdod_0-5cm_mean', 'bdod_100-200cm_mean', 'bdod_15-30cm_mean',
       'bdod_30-60cm_mean', 'bdod_5-15cm_mean', 'bdod_60-100cm_mean','nitrogen_0-5cm_mean', 'nitrogen_100-200cm_mean',
       'nitrogen_15-30cm_mean', 'nitrogen_30-60cm_mean',
       'nitrogen_5-15cm_mean', 'nitrogen_60-100cm_mean']].apply(lambda x: x/100)

In [75]:
data_set_soil_grids[['cec_0-5cm_mean', 'cec_100-200cm_mean', 'cec_15-30cm_mean',
    'cec_30-60cm_mean', 'cec_5-15cm_mean', 'cec_60-100cm_mean','cfvo_0-5cm_mean', 'cfvo_100-200cm_mean', 'cfvo_15-30cm_mean',
    'cfvo_30-60cm_mean', 'cfvo_5-15cm_mean', 'cfvo_60-100cm_mean','clay_0-5cm_mean', 'clay_100-200cm_mean', 'clay_15-30cm_mean',
       'clay_30-60cm_mean', 'clay_5-15cm_mean', 'clay_60-100cm_mean','phh2o_0-5cm_mean',
       'phh2o_100-200cm_mean', 'phh2o_15-30cm_mean', 'phh2o_30-60cm_mean',
       'phh2o_5-15cm_mean', 'phh2o_60-100cm_mean','sand_0-5cm_mean',
       'sand_100-200cm_mean', 'sand_15-30cm_mean', 'sand_30-60cm_mean',
       'sand_5-15cm_mean', 'sand_60-100cm_mean', 'silt_0-5cm_mean',
       'silt_100-200cm_mean', 'silt_15-30cm_mean', 'silt_30-60cm_mean',
       'silt_5-15cm_mean', 'silt_60-100cm_mean', 'soc_0-5cm_mean',
       'soc_100-200cm_mean', 'soc_15-30cm_mean', 'soc_30-60cm_mean',
       'soc_5-15cm_mean', 'soc_60-100cm_mean']] = data_set_soil_grids[['cec_0-5cm_mean', 'cec_100-200cm_mean', 'cec_15-30cm_mean',
    'cec_30-60cm_mean', 'cec_5-15cm_mean', 'cec_60-100cm_mean','cfvo_0-5cm_mean', 'cfvo_100-200cm_mean', 'cfvo_15-30cm_mean',
    'cfvo_30-60cm_mean', 'cfvo_5-15cm_mean', 'cfvo_60-100cm_mean','clay_0-5cm_mean', 'clay_100-200cm_mean', 'clay_15-30cm_mean',
       'clay_30-60cm_mean', 'clay_5-15cm_mean', 'clay_60-100cm_mean','phh2o_0-5cm_mean',
       'phh2o_100-200cm_mean', 'phh2o_15-30cm_mean', 'phh2o_30-60cm_mean',
       'phh2o_5-15cm_mean', 'phh2o_60-100cm_mean','sand_0-5cm_mean',
       'sand_100-200cm_mean', 'sand_15-30cm_mean', 'sand_30-60cm_mean',
       'sand_5-15cm_mean', 'sand_60-100cm_mean', 'silt_0-5cm_mean',
       'silt_100-200cm_mean', 'silt_15-30cm_mean', 'silt_30-60cm_mean',
       'silt_5-15cm_mean', 'silt_60-100cm_mean', 'soc_0-5cm_mean',
       'soc_100-200cm_mean', 'soc_15-30cm_mean', 'soc_30-60cm_mean',
       'soc_5-15cm_mean', 'soc_60-100cm_mean']].apply(lambda x: x/10) 

In [76]:
data_set_soil_grids.to_excel('DB_soilgrids_conventional_units.xlsx')